In [1]:
# NLP
# import nltk
# nltk.download()

In [2]:
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import linear_kernel

In [3]:
# load products
df = pd.read_json('products.json', orient='records')

In [4]:
df.head()

,product_code,product_name,product_price,product_category,product_description,product_more_info,product_image
0,00007,Contorno De Ojos En Gel Hidratante,200,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...
1,01176,Crema Facial Exfoliante Aclarante,290,faciales,Exfoliación suave que elimina a diario las cél...,LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,https://www.yvesrocher.com.mx/Areas/YR/images/...
2,01340,Maxi Gel Facial Limpiador Hidratante,340,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...
3,06043,Mascarilla Facial Nocturna Aclarante,359,faciales,"Su piel es suave, hidratada y radiante después...",LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,https://www.yvesrocher.com.mx/Areas/YR/images/...
4,11783,Crema Facial Suavizante Día/noche,175,faciales,"Hidrata,suaviza y protege la piel. Su plus:\nT...","MANZANILLA En 1964, Yves Rocher imaginó por pr...",https://www.yvesrocher.com.mx/Areas/YR/images/...


In [5]:
# create a column with all words from text columns
text_cols = ['product_name', 'product_category', 'product_description', 'product_more_info']
df['text_processed'] = df[text_cols].apply(lambda x: " ".join(x), axis=1)
df.head()

,product_code,product_name,product_price,product_category,product_description,product_more_info,product_image,text_processed
0,00007,Contorno De Ojos En Gel Hidratante,200,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...,Contorno De Ojos En Gel Hidratante faciales Nu...
1,01176,Crema Facial Exfoliante Aclarante,290,faciales,Exfoliación suave que elimina a diario las cél...,LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,https://www.yvesrocher.com.mx/Areas/YR/images/...,Crema Facial Exfoliante Aclarante faciales Exf...
2,01340,Maxi Gel Facial Limpiador Hidratante,340,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...,Maxi Gel Facial Limpiador Hidratante faciales ...
3,06043,Mascarilla Facial Nocturna Aclarante,359,faciales,"Su piel es suave, hidratada y radiante después...",LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,https://www.yvesrocher.com.mx/Areas/YR/images/...,Mascarilla Facial Nocturna Aclarante faciales ...
4,11783,Crema Facial Suavizante Día/noche,175,faciales,"Hidrata,suaviza y protege la piel. Su plus:\nT...","MANZANILLA En 1964, Yves Rocher imaginó por pr...",https://www.yvesrocher.com.mx/Areas/YR/images/...,Crema Facial Suavizante Día/noche faciales Hid...


In [6]:
stop_words = stopwords.words('spanish')
stemmer = SnowballStemmer("spanish", ignore_stopwords=True)

def clean_up(s):
    word_list = s.lower().split(" ")
    word_list = [re.sub(r'\d', ' ', word) for word in word_list]
    word_list = [re.sub(r'\W', ' ', word) for word in word_list]
    new_string = " ".join(word_list)

    token_list = word_tokenize(new_string)
    stem_list=[stemmer.stem(token) for token in token_list]
    new_list = [token for token in stem_list if token not in stop_words]
    new_string = " ".join(new_list)
    return new_string

In [7]:
df['text_processed']=df['text_processed'].apply(clean_up)
df.head()

,product_code,product_name,product_price,product_category,product_description,product_more_info,product_image,text_processed
0,00007,Contorno De Ojos En Gel Hidratante,200,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...,contorn ojos gel hidrat facial expert cosmet v...
1,01176,Crema Facial Exfoliante Aclarante,290,faciales,Exfoliación suave que elimina a diario las cél...,LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,https://www.yvesrocher.com.mx/Areas/YR/images/...,crem facial exfoli aclar facial exfoli suav el...
2,01340,Maxi Gel Facial Limpiador Hidratante,340,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...,maxi gel facial limpiador hidrat facial expert...
3,06043,Mascarilla Facial Nocturna Aclarante,359,faciales,"Su piel es suave, hidratada y radiante después...",LUPINO BLANCO Y REGALIZ Nuestros expertos en I...,https://www.yvesrocher.com.mx/Areas/YR/images/...,mascarill facial nocturn aclar facial piel sua...
4,11783,Crema Facial Suavizante Día/noche,175,faciales,"Hidrata,suaviza y protege la piel. Su plus:\nT...","MANZANILLA En 1964, Yves Rocher imaginó por pr...",https://www.yvesrocher.com.mx/Areas/YR/images/...,crem facial suaviz dia noch facial hidrat suav...


In [8]:
# Define a TF-IDF Vectorizer Object
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=0.003
)

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = vectorizer.fit_transform(df['text_processed'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(621, 6959)

7,339 different words were used to describe the 655 products in our dataset.

We can now compute a similarity score. We will be using the cosine similarity to calculate a numeric quantity that denotes the similarity between two products.

Since we have used the TF-IDF vectorizer, calculating the dot product will directly give us the cosine similarity score. Therefore, we will use sklearn's `linear_kernel()` instead of `cosine_similarities()`

In [9]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [10]:
#Construct a reverse map of indices and product codes
indices = pd.Series(df.index, index=df.product_code).drop_duplicates()

In [11]:
# Function that takes in product code as input and outputs most similar product
def get_recommendation(product_id, similarity=cosine_sim):
    # Get the index of the product that matches the product_id
    idx = indices[product_id]
    
    # Get the pairwsie similarity scores of all products with that product
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the products based on the scores
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    
    # Get the score of the 10 most similar products
    sim_scores = sim_scores[1:11]
    
    # Get the product indices
    product_indices = [i[0] for i in sim_scores]
    
    # Return  the top most similar movies
    return df.iloc[product_indices]

In [12]:
# ['57842', '73013', '34725', '56317', '20700']

In [13]:
get_recommendation('63866')

,product_code,product_name,product_price,product_category,product_description,product_more_info,product_image,text_processed
184,77403,Maxi Leche Corporal Nutritiva Piel Muy Seca,299,corporales,¿Deseas nutrir y suavizar la piel muy seca?La ...,AVENA El secreto de la Avena: la vida. La semi...,https://www.yvesrocher.com.mx/Areas/YR/images/...,maxi lech corporal nutrit piel sec corporal de...
563,34733,Bálsamo De Ducha Nutritivo,199,higiene,Nutre y Suaviza la piel durante la ducha. Nues...,AVENA El secreto de la Avena: la vida. La semi...,https://www.yvesrocher.com.mx/Areas/YR/images/...,balsam duch nutrit higien nutr suaviz piel duc...
158,53022,Crema Corporal Hidratante Anticelulitis,305,corporales,"HIDRATA, EMBELLECE LA PIEL Y REDUCE EL ASPECTO...",MANGOSTÁN Gracias a un proceso de extracción 1...,https://www.yvesrocher.com.mx/Areas/YR/images/...,crem corporal hidrat anticelulitis corporal hi...
139,12989,Leche Corporal Hidratante 2 En 1 Efecto Firmeza,179,corporales,Nuestros Expertos en Cosmética Vegetal han sel...,CENTELLA ASIÁTICA Los Investigadores en Cosmét...,https://www.yvesrocher.com.mx/Areas/YR/images/...,lech corporal hidrat efect firmez corporal exp...
155,42135,Leche Corporal Reparadora Piel Extra Seca,229,corporales,Con esta leche tu piel queda intensamente nutr...,LA MANTECA DE KARITÉ Nuestros Investigadores e...,https://www.yvesrocher.com.mx/Areas/YR/images/...,lech corporal repar piel extra sec corporal le...
156,42517,Maxi Leche Corporal Reparadora Piel Extra Seca,299,corporales,Con esta leche tu piel queda intensamente nutr...,LA MANTECA DE KARITÉ Nuestros Investigadores e...,https://www.yvesrocher.com.mx/Areas/YR/images/...,maxi lech corporal repar piel extra sec corpor...
170,62259,Leche Corporal Hidratante Piel Seca,229,corporales,¿Deseas hidratar y reforzar intensamente tu pi...,ALOE VERA El secreto del Aloe Vera: la adaptac...,https://www.yvesrocher.com.mx/Areas/YR/images/...,lech corporal hidrat piel sec corporal des hid...
183,75501,Maxi Leche Corporal Hidratante Piel Seca,299,corporales,¿Deseas hidratar y reforzar intensamente tu pi...,ALOE VERA El secreto del Aloe Vera: la adaptac...,https://www.yvesrocher.com.mx/Areas/YR/images/...,maxi lech corporal hidrat piel sec corporal de...
16,19249,Gel Exfoliante Anticontaminación,290,faciales,Nuestros investigadores en Cosmética Vegetal h...,APHLOIA: Nuestros Expertos en Cosmética Vegeta...,https://www.yvesrocher.com.mx/Areas/YR/images/...,gel exfoli anticontamin facial investig cosmet...
2,01340,Maxi Gel Facial Limpiador Hidratante,340,faciales,Nuestros Expertos en Cosmética Vegetal han sel...,EDULIS: Nuestros Investigadores en Cosmética V...,https://www.yvesrocher.com.mx/Areas/YR/images/...,maxi gel facial limpiador hidrat facial expert...


In [52]:
my_test = '''
aroma fresco'''

In [53]:
new_test = clean_up(my_test)
new_test

'arom fresc'

In [54]:
otra_matriz = vectorizer.transform(
    pd.concat([df['text_processed'],
               pd.Series([new_test])], 
              ignore_index=True)
)

In [55]:
simi_otra = linear_kernel(otra_matriz, otra_matriz)

In [56]:
lista_otra = sorted(list(enumerate(simi_otra[-1])), key=lambda x:x[1], reverse=True)[1:11]


In [57]:
una_mas = [i[0] for i in lista_otra]
una_mas

[99, 102, 103, 506, 516, 171, 72, 497, 162, 151]

In [58]:
df.iloc[una_mas]

,product_code,product_name,product_price,product_category,product_description,product_more_info,product_image,text_processed
99,40291,Jumbo Shampoo Brillo Espectacular,249,capilares,El Shampoo Brillo Espectácular limpia suavemen...,EXTRACTO DE LIMÓN Y VITAMINA VEGETAL Aportan u...,https://www.yvesrocher.com.mx/Areas/YR/images/...,jumb shampo brill espectacul capilar shampo br...
102,42879,Shampoo Brillo Espectacular,109,capilares,El Shampoo Brillo Espectácular limpia suavemen...,EXTRACTO DE LIMÓN Y VITAMINA VEGETAL Aportan u...,https://www.yvesrocher.com.mx/Areas/YR/images/...,shampo brill espectacul capilar shampo brill e...
103,43680,Eco Shampoo Concentrado Brillo Espectacular,89,capilares,"""CABELLOS SUAVES Y BRILLANTES\nLa fórmula conc...","""EXTRACTO DE LIMÓN Y VITAMINA VEGETAL Aportan ...",https://www.yvesrocher.com.mx/Areas/YR/images/...,eco shampo concentr brill espectacul capilar c...
506,44312,Agua De Perfume Tropicale Tentation,439,fragancias,El Agua de Perfume Tropicale Tentation es el p...,NOTAS OLFATIVAS: FLOR DORADA DE OSMANTHUS Y NA...,https://www.yvesrocher.com.mx/Areas/YR/images/...,agu perfum tropical tentation fraganci agu per...
516,47112,Mini Agua De Perfume Tropicale Tentation,359,fragancias,El Agua de Perfume Tropicale Tentation es el p...,NOTAS OLFATIVAS: FLOR DORADA DE OSMANTHUS Y NA...,https://www.yvesrocher.com.mx/Areas/YR/images/...,mini agu perfum tropical tentation fraganci ag...
171,62501,Crema Para Manos Tropicale Tentation,79,corporales,Descubre tu fragancia favorita en esta suave y...,NOTAS OLFATIVAS: FLOR DORADA DE OSMANTHUS Y NA...,https://www.yvesrocher.com.mx/Areas/YR/images/...,crem man tropical tentation corporal descubr f...
72,67014,Contorno De Ojos Efecto Lifting,285,faciales,"Este gel de textura fresca hidrata, reduce las...","BÚGULA: La Ajuga Reptans, también conocida com...",https://www.yvesrocher.com.mx/Areas/YR/images/...,contorn ojos efect lifting facial gel textur f...
497,37086,Agua De Tocador Comme Une Evidence Homme,349,fragancias,Nuestro Vetiver haitiano garantiza un sumistro...,"NOTAS OLFATIVAS: Vetiver, Gaiac, Bergamota, Pi...",https://www.yvesrocher.com.mx/Areas/YR/images/...,agu tocador comm une evidenc homm fraganci vet...
162,55561,Crema Corporal Fluida Mango Y Cilantro,149,corporales,Para el placer de un tratamiento corporal ener...,MANGO Y CILANTRO Yves Rocher ha seleccionado e...,https://www.yvesrocher.com.mx/Areas/YR/images/...,crem corporal flu mang cilantr corporal plac t...
151,31253,Maxi Crema Corporal Fluida Mango Y Cilantro,119,corporales,Para el placer de un tratamiento corporal ener...,MANGO Y CILANTRO Yves Rocher ha seleccionado e...,https://www.yvesrocher.com.mx/Areas/YR/images/...,maxi crem corporal flu mang cilantr corporal p...
